In [1]:
import pandas as pd
import gc
import re
import numpy as np
import torch
from imblearn.over_sampling import RandomOverSampler
import datasets
import transformers
print(transformers.__version__)
from pathlib import Path
import torchaudio

import warnings 
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

print("Is GPU available? ", torch.cuda.is_available())
print("GPU Name: ", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

import os

# Set the environment variable
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

4.46.3
Is GPU available?  True
GPU Name:  NVIDIA GeForce RTX 4090


In [2]:
RATE_HZ = 16000 # resampling rate in Hz
MAX_LENGTH = 250000 # maximum audio interval length to consider (= RATE_HZ * SECONDS)
AUDIO_FOLDER = Path("fma_small") 

In [3]:
import json
# this for large label mapping file
# lableid_file = 'mappings.json'

# # Read mappings back from the JSON file
# with open(lableid_file, 'r') as f:
#     mappings = json.load(f)
#     label2id = mappings['label2id']
#     id2label = mappings['id2label']

# # Print loaded mappings to confirm
# print("Loaded label2id:", label2id)
# print("Loaded id2label:", id2label)
# print(len(label2id))
labels=['Hip-Hop','Pop','Folk','Experimental','Rock','International','Electronic','Instrumental']
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label
print(id2label, '\n\n', label2id)

{0: 'Hip-Hop', 1: 'Pop', 2: 'Folk', 3: 'Experimental', 4: 'Rock', 5: 'International', 6: 'Electronic', 7: 'Instrumental'} 

 {'Hip-Hop': 0, 'Pop': 1, 'Folk': 2, 'Experimental': 3, 'Rock': 4, 'International': 5, 'Electronic': 6, 'Instrumental': 7}


In [4]:
from pathlib import Path
import torchaudio
# this is for all tracks in FMA 
# with open('track_genres.json', 'r') as f:
#     data = json.load(f)
# df = [{"file": track_id, "label": genre_id} for track_id, genre_id in data.items()]
# dd = pd.DataFrame(df)
# print(dd.head())
# print(dd.shape)
# print(dd['label'].value_counts())
df1=pd.read_csv("track2genre_8000.csv")
print(df1.head())
df1['genre']=df1['genre'].map(label2id)
print(df1.head())
df1.columns=['file','label']

   track_id    genre
0         2  Hip-Hop
1         5  Hip-Hop
2        10      Pop
3       140     Folk
4       141     Folk
   track_id  genre
0         2      0
1         5      0
2        10      1
3       140      2
4       141      2
   file  label
0     2      0
1     5      0
2    10      1
3   140      2
4   141      2


In [5]:
def get_transform_audio(file):
    audio,rate = torchaudio.load(str(file))
    transform = torchaudio.transforms.Resample(rate,RATE_HZ)
    if audio.size(0) > 1:  # 如果是多通道
        audio = audio.mean(dim=0)
    audio = transform(audio).numpy()
    audio = audio[:MAX_LENGTH]
    return audio # truncate to first part of audio to save RAM

In [6]:
audio_data = []
labels = []

for idx, row in df1.iterrows():
    track_id = str(row["file"]).zfill(6)
    label = row["label"]
    audio_path = AUDIO_FOLDER / track_id[:3] / f"{track_id}.mp3"
    
    if audio_path.exists():
        try:
            audio = get_transform_audio(audio_path)
            audio_data.append(audio)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {audio_path}: {e}")
    # else:
    #     print(f"not found: {audio_path}")
    if (idx + 1) % 200 == 0:
        print(f"Processed {idx + 1} tracks")

# 检查结果
print("Processed audio files:", len(audio_data))
print("Processed labels:", len(labels))

Processed 200 tracks
Processed 400 tracks


[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!


Processed 600 tracks
Processed 800 tracks


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


Processed 1000 tracks


...

Error processing fma_small/099/099134.mp3: Error opening 'fma_small/099/099134.mp3': File does not exist or is not a regular file (possibly a pipe?).
Processed 4600 tracks
Processed 4800 tracks


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...


Error processing fma_small/108/108925.mp3: Error opening 'fma_small/108/108925.mp3': File does not exist or is not a regular file (possibly a pipe?).
Processed 5000 tracks
Processed 5200 tracks
Processed 5400 tracks
Processed 5600 tracks
Processed 5800 tracks
Processed 6000 tracks
Processed 6200 tracks
Processed 6400 tracks
Processed 6600 tracks
Processed 6800 tracks


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...


Error processing fma_small/133/133297.mp3: Error opening 'fma_small/133/133297.mp3': File does not exist or is not a regular file (possibly a pipe?).
Processed 7000 tracks
Processed 7200 tracks
Processed 7400 tracks
Processed 7600 tracks
Processed 7800 tracks
Processed 8000 tracks
Processed audio files: 7994
Processed labels: 7994


In [7]:
print(type(labels))
CLASS_NUM=len(np.unique(labels))
print(CLASS_NUM)
labels = torch.tensor(labels, dtype=torch.long)
print("Tensor dtype:", labels.dtype)

totdd = pd.DataFrame({
    "label": labels,
    "audio": audio_data
})
# check the data
totdd["audio"] = totdd["audio"].apply(lambda x: np.array(x, dtype=np.float32))
print(totdd.sample(5)) 
print(totdd['label'].dtype)

<class 'list'>
8
Tensor dtype: torch.int64
      label                                              audio
6585      6  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6210      2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
7858      4  [0.04892895, 0.08216659, 0.09265838, 0.1291345...
5450      7  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3126      1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
      label                                              audio
6383      3  [-0.021570303, -0.040477242, -0.039989594, -0....
5381      6  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
6033      1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1636      7  [0.007150736, 0.018054802, -0.17563047, -0.217...
3042      6  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
int64


In [8]:
lengths = totdd["audio"].apply(len)
print(lengths.describe())

count      7994.00000
mean     247341.76695
std       25643.16009
min           1.00000
25%      250000.00000
50%      250000.00000
75%      250000.00000
max      250000.00000
Name: audio, dtype: float64
0    250000
1    250000
2    250000
3    250000
4    250000
Name: audio, dtype: int64


In [9]:
print(totdd["audio"].apply(len).nunique()) 
totdd = totdd[totdd["audio"].apply(len) >= 16000]
print(totdd["audio"].apply(len).nunique()) 

2
1


In [10]:
from datasets import Dataset, ClassLabel
totdd = Dataset.from_pandas(totdd)

from collections import Counter
Counter(totdd['label']).items()


dict_items([(0, 992), (1, 997), (2, 985), (3, 971), (4, 994), (5, 992), (6, 978), (7, 1000)])

In [11]:
print(type(totdd['label'][0]))
totdd = totdd.map(lambda x: {"label": torch.tensor(x["label"], dtype=torch.long)}, batched=True)
print(type(totdd['label'][0]))

<class 'int'>


Map:   0%|          | 0/7909 [00:00<?, ? examples/s]

<class 'int'>


In [12]:
totdd = totdd.train_test_split(test_size=0.2)

In [13]:
print(totdd)
totdd['train'] = totdd['train'].remove_columns(['__index_level_0__'])
totdd['test'] = totdd['test'].remove_columns(['__index_level_0__'])
totdd

DatasetDict({
    train: Dataset({
        features: ['label', 'audio', '__index_level_0__'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['label', 'audio', '__index_level_0__'],
        num_rows: 1582
    })
})


DatasetDict({
    train: Dataset({
        features: ['label', 'audio'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['label', 'audio'],
        num_rows: 1582
    })
})

In [14]:
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification

model_str = "MIT/ast-finetuned-audioset-10-10-0.4593" 
feature_extractor = AutoFeatureExtractor.from_pretrained(model_str)
model = AutoModelForAudioClassification.from_pretrained(model_str,num_labels=len(label2id),ignore_mismatched_sizes=True)
model.config.id2label = id2label
# number of trainable parameters
print(model.num_parameters(only_trainable=True)/1e6)
print(len(label2id))
print(len(np.unique(labels)))

preprocessor_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.51k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at dima806/music_genres_classification and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([10]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([10, 256]) in the checkpoint and torch.Size([8, 256]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


94.570632
8
8


In [15]:
def preprocess_function(batch):    
    inputs = feature_extractor(
        batch['audio'], 
        sampling_rate=RATE_HZ, 
        max_length=MAX_LENGTH, 
        truncation=True
    )
    inputs['input_values'] = inputs['input_values'][0]
    
    # Convert label to torch.long
    if 'label' in batch:
        inputs['labels'] = torch.tensor(batch['label'], dtype=torch.long)
    
    return inputs

totdd['test'] = totdd['test'].map(
    preprocess_function, 
    remove_columns=totdd['test'].column_names,
    batched=False
)

totdd['train'] = totdd['train'].map(
    preprocess_function, 
    remove_columns=totdd['train'].column_names,
    batched=False
)
totdd['train'].set_format(type='torch', columns=['input_values', 'labels'])
totdd['test'].set_format(type='torch', columns=['input_values', 'labels'])

Map:   0%|          | 0/1582 [00:00<?, ? examples/s]

Map:   0%|          | 0/6327 [00:00<?, ? examples/s]

In [16]:
totdd.save_to_disk("fma_small_class8_wave2vec")

Saving the dataset (0/13 shards):   0%|          | 0/6327 [00:00<?, ? examples/s]

Saving the dataset (0/4 shards):   0%|          | 0/1582 [00:00<?, ? examples/s]

In [17]:
print("Training data label type:", type(totdd['train'][0]['labels']))
print("Training data label dtype:", totdd['train'][0]['labels'].dtype)
print("Test data label type:", type(totdd['test'][0]['labels']))
print("Test data label dtype:", totdd['test'][0]['labels'].dtype)

Training data label type: <class 'torch.Tensor'>
Training data label dtype: torch.int64
Test data label type: <class 'torch.Tensor'>
Test data label dtype: torch.int64


In [18]:
gc.collect()

226

In [19]:
import evaluate
from sklearn.preprocessing import label_binarize

accuracy = evaluate.load("accuracy")

from sklearn.metrics import roc_auc_score
def compute_metrics(eval_pred):
    predictions = eval_pred.predictions  
    label_ids = eval_pred.label_ids    
    
    present_classes = np.unique(label_ids)
    # print(f"Present classes in current dataset: {present_classes}")
    # print(f"Shape of predictions: {predictions.shape}")
    
    predictions = np.exp(predictions)/np.exp(predictions).sum(axis=1, keepdims=True)
    
    acc_score = accuracy.compute(
        predictions=predictions.argmax(axis=1),
        references=label_ids
    )['accuracy']
    
    y_true_bin = label_binarize(label_ids, classes=present_classes)
    
    predictions_subset = predictions[:, present_classes]
    
    try:
        roc_auc = roc_auc_score(
            y_true=y_true_bin,
            y_score=predictions_subset,
            multi_class='ovr',
            average='macro'
        )
    except ValueError as e:
        print(f"ROC AUC calculation error: {e}")
        roc_auc = 0.0
    
    return {
        "roc_auc": roc_auc,
        "accuracy": acc_score,
        "present_classes": len(present_classes)
    }

In [20]:
print("Train labels:", totdd["train"]["labels"].unique())
print("Test labels:", totdd["test"]["labels"].unique())

Train labels: tensor([0, 1, 2, 3, 4, 5, 6, 7])
Test labels: tensor([0, 1, 2, 3, 4, 5, 6, 7])


In [21]:
from transformers import TrainingArguments, Trainer
batch_size=5
warmup_steps=50
weight_decay=0.02
num_train_epochs=10
model_name = "fma_small_finetuned_1.0_AST"
training_args = TrainingArguments(
    output_dir=model_name,
    logging_dir='./logs',
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=5e-5, # 5e-6
    logging_strategy='steps',
    logging_first_step=True,
    load_best_model_at_end=True,
    logging_steps=10,#1
    evaluation_strategy='epoch',
    warmup_steps=warmup_steps,
    weight_decay=weight_decay,
    eval_steps=20,#1
    gradient_accumulation_steps=1, 
    gradient_checkpointing=True,
    save_strategy='epoch',
    save_total_limit=1, # save fewer checkpoints to limit used space
    report_to="mlflow",  # log to mlflow

    optim="adamw_hf",  # Optimizer, in this case Adam
    adam_beta1=0.9,  # Adam optimizer beta1
    adam_beta2=0.999,  # Adam optimizer beta2
    adam_epsilon=1e-8,  # Adam optimizer epsilon
    fp16=True,
    lr_scheduler_type="linear"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=totdd["train"],
    eval_dataset=totdd["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [22]:
trainer.evaluate()

{'eval_loss': 2.1797096729278564,
 'eval_model_preparation_time': 0.0032,
 'eval_roc_auc': 0.43770623958633986,
 'eval_accuracy': 0.07901390644753477,
 'eval_present_classes': 8,
 'eval_runtime': 24.3378,
 'eval_samples_per_second': 65.002,
 'eval_steps_per_second': 13.025}

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Roc Auc,Accuracy,Present Classes
1,2.011400,1.928674,0.003200,0.655313,0.213654,8
2,2.082900,2.083765,0.003200,0.494915,0.125158,8
3,2.066600,2.075129,0.003200,0.533946,0.125790,8
4,2.105300,2.082439,0.003200,0.507632,0.118837,8
5,2.083400,2.079763,0.003200,0.512423,0.127686,8
6,2.091400,2.080866,0.003200,0.501181,0.118837,8
7,2.080600,2.079350,0.003200,0.494039,0.123262,8
8,2.083800,2.080013,0.003200,0.512517,0.118837,8
9,2.081100,2.080103,0.003200,0.503930,0.118837,8
10,2.083300,2.080346,0.003200,0.499573,0.118837,8


TrainOutput(global_step=12660, training_loss=2.064557850247876, metrics={'train_runtime': 2527.9804, 'train_samples_per_second': 25.028, 'train_steps_per_second': 5.008, 'total_flos': 8.97522582996031e+18, 'train_loss': 2.064557850247876, 'epoch': 10.0})

In [24]:
trainer.evaluate()

{'eval_loss': 1.9286738634109497,
 'eval_model_preparation_time': 0.0032,
 'eval_roc_auc': 0.6553131521179113,
 'eval_accuracy': 0.213653603034134,
 'eval_present_classes': 8,
 'eval_runtime': 22.146,
 'eval_samples_per_second': 71.435,
 'eval_steps_per_second': 14.314,
 'epoch': 10.0}

In [25]:
trainer.save_model('for_infere')

In [26]:
from transformers import pipeline

pipe=pipeline('audio-classification',model='for_infere',device=0,return_all_scores=True)
audio,rate=torchaudio.load('fma_small/032/032330.mp3')
transform=torchaudio.transforms.Resample(rate,RATE_HZ)
audio1=transform(audio).numpy().reshape(-1)
# make a classification pipeline
pipe(audio1)

[{'score': 0.26550349593162537, 'label': 'Folk'},
 {'score': 0.15864621102809906, 'label': 'Rock'},
 {'score': 0.13659189641475677, 'label': 'International'},
 {'score': 0.12934762239456177, 'label': 'Experimental'},
 {'score': 0.1281781941652298, 'label': 'Instrumental'}]

In [27]:
# logitsmodel = AutoModelForAudioClassification.from_pretrained('for_infere')
# from transformers import AutoProcessor
# processor = AutoProcessor.from_pretrained('for_infere')
# audio2=transform(audio).squeeze(0) 
# inputs = processor(audio2, sampling_rate=RATE_HZ, return_tensors="pt",padding=True)
# with torch.no_grad():
#     logits = logitsmodel(**inputs).logits
# logits

In [28]:
trainer.save_model('model_save_path')